# 09 향후 판매량 예측

## 9.4 성능 개선

In [352]:
import gc

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

In [353]:
# 데이터 블로오기
data_path = '../../data/09_sales/'

sales_train = pd.read_csv(data_path + 'sales_train.csv')
shops = pd.read_csv(data_path + 'shops.csv')
items = pd.read_csv(data_path + 'items.csv')
item_categories = pd.read_csv(data_path + 'item_categories.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

### 9.4.1 피처 엔지니어링I: 피처명 한글화와 데이터 다운캐스팅

In [354]:
sales_train = sales_train.rename(columns={
    'date': '날짜',
    'date_block_num': '월ID',
    'shop_id': '상점ID',
    'item_id': '상품ID',
    'item_price': '판매가',
    'item_cnt_day': '판매량',
})
shops = shops.rename(columns={
    'shop_name': '상점명',
    'shop_id': '상점ID',
})
items = items.rename(columns={
    'item_name': '상품명',
    'item_id': '상품ID',
    'item_category_id': '상품분류ID',
})
item_categories = item_categories.rename(columns={
    'item_category_name': '상품분류명',
    'item_category_id': '상품분류ID',
})
test = test.rename(columns={
    'shop_id': '상점ID',
    'item_id': '상품ID',
})

In [355]:
def downcast(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        dtype_name = df[col].dtype.name
        if dtype_name == 'object':
            pass
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
        elif dtype_name.startswith('int') or (df[col].round() == df[col]).all():
            df[col] = pd.to_numeric(df[col], downcast='integer')
        else:
            df[col] = pd.to_numeric(df[col], downcast='float')
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print(f'{100*(start_mem-end_mem)/start_mem:.2f} % 압축됨')
    return df

In [356]:
all_df = [sales_train, shops, items, item_categories, test]
for df in all_df:
    df = downcast(df)

54.17 % 압축됨
38.60 % 압축됨
54.15 % 압축됨
39.95 % 압축됨
70.83 % 압축됨


### 9.4.2 피처 엔지니어링II: 개별 데이터 피처 엔지니어링

In [357]:
# sales_train 이상치 제거 및 전처리
sales_train = sales_train[sales_train['판매가'] > 0]
sales_train = sales_train[sales_train['판매가'] < 50000]

sales_train = sales_train[sales_train['판매량'] > 0]
sales_train = sales_train[sales_train['판매량'] < 1000]

In [358]:
# 상점ID 누락 처리
sales_train.loc[sales_train['상점ID'] == 0, '상점ID'] = 57
sales_train.loc[sales_train['상점ID'] == 1, '상점ID'] = 58
sales_train.loc[sales_train['상점ID'] == 10, '상점ID'] = 11
sales_train.loc[sales_train['상점ID'] == 39, '상점ID'] = 40

In [359]:
# test 데이터에서 상점 ID 수정
test.loc[test['상점ID'] == 0, '상점ID'] = 57
test.loc[test['상점ID'] == 1, '상점ID'] = 58
test.loc[test['상점ID'] == 10, '상점ID'] = 11
test.loc[test['상점ID'] == 39, '상점ID'] = 40

In [360]:
# shops 파생 피처 생성 및 인코딩
shops['도시'] = shops['상점명'].apply(lambda x: x.split()[0])
shops['도시'].unique()

array(['!Якутск', 'Адыгея', 'Балашиха', 'Волжский', 'Вологда', 'Воронеж',
       'Выездная', 'Жуковский', 'Интернет-магазин', 'Казань', 'Калуга',
       'Коломна', 'Красноярск', 'Курск', 'Москва', 'Мытищи', 'Н.Новгород',
       'Новосибирск', 'Омск', 'РостовНаДону', 'СПб', 'Самара', 'Сергиев',
       'Сургут', 'Томск', 'Тюмень', 'Уфа', 'Химки', 'Цифровой', 'Чехов',
       'Якутск', 'Ярославль'], dtype=object)

In [361]:
shops.loc[shops['도시']=='!Якутск', '도시'] = 'Якутск'

In [362]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
shops['도시'] = label_encoder.fit_transform(shops['도시'])

In [363]:
shops = shops.drop(columns='상점명')
shops.head()

,상점ID,도시
0,0,29
1,1,29
2,2,0
3,3,1
4,4,2


In [364]:
items = items.drop(columns='상품명')

In [365]:
# items 파생 피처 생성
items['첫 판매월'] = sales_train.groupby('상품ID').agg({'월ID':'min'})['월ID']
items.head()

,상품ID,상품분류ID,첫 판매월
0,0,40,20.0
1,1,76,15.0
2,2,40,19.0
3,3,40,18.0
4,4,40,20.0


In [366]:
items[items['첫 판매월'].isna()]

,상품ID,상품분류ID,첫 판매월
83,83,40,NaN
140,140,45,NaN
168,168,44,NaN
173,173,45,NaN
204,204,44,NaN
...,...,...,...
21974,21974,61,NaN
21975,21975,61,NaN
22022,22022,40,NaN
22035,22035,40,NaN


In [367]:
items['첫 판매월'] = items['첫 판매월'].fillna(34)

In [368]:
# item_categories 파생 피처 생성 및 인코딩
item_categories['대분류'] = item_categories['상품분류명'].apply(lambda x: x.split()[0])

In [369]:
item_categories['대분류'].value_counts()

대분류
Игры          14
Книги         13
Подарки       12
Игровые        8
Аксессуары     7
Музыка         6
Программы      6
Карты          5
Кино           5
Служебные      2
Чистые         2
PC             1
Билеты         1
Доставка       1
Элементы       1
Name: count, dtype: int64

In [370]:
def make_etc(x):
    if len(item_categories[item_categories['대분류']==x]) >= 5:
        return x
    else:
        return 'etc'

item_categories['대분류'] = item_categories['대분류'].apply(make_etc)
item_categories.head()

,상품분류명,상품분류ID,대분류
0,PC - Гарнитуры/Наушники,0,etc
1,Аксессуары - PS2,1,Аксессуары
2,Аксессуары - PS3,2,Аксессуары
3,Аксессуары - PS4,3,Аксессуары
4,Аксессуары - PSP,4,Аксессуары


In [371]:
label_encoder = LabelEncoder()
item_categories['대분류'] = label_encoder.fit_transform(item_categories['대분류'])

In [372]:
item_categories = item_categories.drop(columns='상품분류명')
item_categories.head()

,상품분류ID,대분류
0,0,0
1,1,1
2,2,1
3,3,1
4,4,1


### 9.4.3 피처 엔지니어링III: 데이터 조합 및 파생 피처 생성

In [373]:
# 데이터 조합
from itertools import product
train = []
for i in sales_train['월ID'].unique():
    all_shop = sales_train.loc[sales_train['월ID']==i, '상점ID'].unique()
    all_item = sales_train.loc[sales_train['월ID']==i, '상점ID'].unique()
    train.append(np.array(list(product([i], all_shop, all_item))))
idx_features = ['월ID', '상점ID', '상품ID']
train = pd.DataFrame(np.vstack(train), columns=idx_features)

In [374]:
# 파생 피처 생성
group = sales_train.groupby(idx_features).agg({'판매량': 'sum', '판매가': 'mean'})
group = group.reset_index()
group = group.rename(columns={'판매량':'월간 판매량', '판매가':'평균 판매가'})
train = train.merge(group, on=idx_features, how='left')
train.head()

,월ID,상점ID,상품ID,월간 판매량,평균 판매가
0,0,59,59,NaN,NaN
1,0,59,25,NaN,NaN
2,0,59,24,NaN,NaN
3,0,59,23,NaN,NaN
4,0,59,19,NaN,NaN


In [375]:
import gc

del group
gc.collect()

8

In [376]:
group = sales_train.groupby(idx_features).agg({'판매량':'count'})
group = group.reset_index()
group = group.rename(columns={'판매량': '판매건수'})
train = train.merge(group, on=idx_features, how='left')
del group, sales_train
gc.collect()

0

In [377]:
train.head(30)

,월ID,상점ID,상품ID,월간 판매량,평균 판매가,판매건수
0,0,59,59,NaN,NaN,NaN
1,0,59,25,NaN,NaN,NaN
2,0,59,24,NaN,NaN,NaN
3,0,59,23,NaN,NaN,NaN
4,0,59,19,NaN,NaN,NaN
5,0,59,22,NaN,NaN,NaN
6,0,59,18,NaN,NaN,NaN
7,0,59,21,NaN,NaN,NaN
8,0,59,28,NaN,NaN,NaN
9,0,59,27,NaN,NaN,NaN


### 9.4.4 피처 엔지니어링IV: 데이터 합치기

In [378]:
test['월ID'] = 34

In [379]:
all_data = pd.concat([train, test.drop(columns='ID')], ignore_index=True, keys=idx_features)
all_data = all_data.fillna(0)
all_data.head()

,월ID,상점ID,상품ID,월간 판매량,평균 판매가,판매건수
0,0,59,59,0.0,0.0,0.0
1,0,59,25,0.0,0.0,0.0
2,0,59,24,0.0,0.0,0.0
3,0,59,23,0.0,0.0,0.0
4,0,59,19,0.0,0.0,0.0


In [380]:
# 모든 데이터 병합
all_data = all_data.merge(shops, on='상점ID', how='left')
all_data = all_data.merge(items, on='상품ID', how='left')
all_data = all_data.merge(item_categories, on='상품분류ID', how='left')

all_data = downcast(all_data)

73.08 % 압축됨


In [381]:
del shops, items, item_categories
gc.collect()

0

### 9.4.5 피처 엔지니어링V: 시차 피처 생성

In [382]:
# 기준 피처별 월간 평균 판매량 파생 피처 생성
def add_mean_features(df, mean_features, idx_features):
    assert (idx_features[0] == '월ID') and len(idx_features) in [2,3]

    if len(idx_features) == 2:
        feature_name = idx_features[1] + '별 평균 판매량'
    else:
        feature_name = idx_features[1] + ' ' + idx_features[2] + '별 평균 판매량'

    group = df.groupby(idx_features).agg({'월간 판매량': 'mean'})
    group = group.reset_index()
    group = group.rename(columns={'월간 판매량': feature_name})

    df = df.merge(group, on=idx_features, how='left')
    df = downcast(df, verbose=False)
    mean_features.append(feature_name)
    del group
    gc.collect()
    return df, mean_features

In [383]:
item_mean_features = []

all_data, item_mean_features = add_mean_features(
    df = all_data,
    mean_features=item_mean_features,
    idx_features=['월ID', '상품ID']
)

all_data, item_mean_features = add_mean_features(
    df = all_data,
    mean_features=item_mean_features,
    idx_features=['월ID', '상품ID', '도시']
)

In [384]:
shop_mean_features = []
all_data, shop_mean_features = add_mean_features(
    df=all_data,
    mean_features=shop_mean_features,
    idx_features=['월ID', '상점ID', '상품분류ID']
)

In [385]:
# 시차 피처 생성 원리 및 함수 구현
def add_lag_features(df, lag_features_to_clip, idx_features, lag_feature, nlags=3, clip=False):
    df_temp = df[idx_features+[lag_feature]].copy()
    for i in range(1, nlags+1):
        lag_feature_name = lag_feature + '_시차' + str(i)
        df_temp.columns = idx_features + [lag_feature_name]
        df_temp['월ID'] += 1
        df = df.merge(df_temp.drop_duplicates(), on=idx_features, how='left')
        df[lag_feature_name] = df[lag_feature_name].fillna(0)
        if clip:
            lag_features_to_clip.append(lag_feature_name)
    df = downcast(df, False)
    del df_temp
    gc.collect()
    return df, lag_features_to_clip

In [386]:
# 시차 피처 생성I : 월간 판매량
lag_features_to_clip = [] # 0 ~ 20 사이로 제한할 시차 피처명을 담을 리스트
idx_features = ['월ID', '상점ID', '상품ID'] # 기준 피처

# idx_features를 기준으로 월간 판매량의 세 달치 시차 피처 생성
all_data, lag_features_to_clip = add_lag_features(df=all_data,
                                                  lag_features_to_clip=lag_features_to_clip,
                                                  idx_features=idx_features,
                                                  lag_feature='월간 판매량',
                                                  nlags=3,
                                                  clip=True) # 값을 0 ~ 20 사이로 제한

In [387]:
# 시차 피처 생성 II: 판매건수, 평균 판매가
# idx_features를 기준으로 판매건수 피처의 세 달치 시차 피처 생성
all_data, lag_features_to_clip = add_lag_features(df=all_data,
                                                  lag_features_to_clip=lag_features_to_clip,
                                                  idx_features=idx_features,
                                                  lag_feature='판매건수',
                                                  nlags=3)

# idx_features를 기준으로 평균 판매가 피처의 세 달치 시차 피처 생성
all_data, lag_features_to_clip = add_lag_features(df=all_data,
                                                  lag_features_to_clip=lag_features_to_clip,
                                                  idx_features=idx_features,
                                                  lag_feature='평균 판매가',
                                                  nlags=3)

In [388]:
# 시차 피처 생성 III: 평균 판매량
# idx_features를 기준으로 item_mean_features 요소별 시차 피처 생성
for item_mean_feature in item_mean_features:
    all_data, lag_features_to_clip = add_lag_features(df=all_data,
                                                      lag_features_to_clip=lag_features_to_clip,
                                                      idx_features=idx_features,
                                                      lag_feature=item_mean_feature,
                                                      nlags=3,
                                                      clip=True)
# item_mean_features 피처 제거
all_data = all_data.drop(item_mean_features, axis=1)

In [391]:
all_data[all_data['월간 판매량_시차2'] != 0].head(50)

,월ID,상점ID,상품ID,월간 판매량,평균 판매가,판매건수,도시,상품분류ID,첫 판매월,대분류,...,판매건수_시차3,평균 판매가_시차1,평균 판매가_시차2,평균 판매가_시차3,상품ID별 평균 판매량_시차1,상품ID별 평균 판매량_시차2,상품ID별 평균 판매량_시차3,상품ID 도시별 평균 판매량_시차1,상품ID 도시별 평균 판매량_시차2,상품ID 도시별 평균 판매량_시차3
4224,2,2,27,0,0.000000,0,0,19,0,3,...,0,0.0,2499.000000,0.0,0.065217,0.155556,0.0,0.000000,1.000000,0.0
4245,2,41,32,2,349.000000,2,18,40,0,5,...,0,349.0,349.000000,0.0,4.521739,6.644444,0.0,1.000000,1.000000,0.0
4291,2,38,32,1,349.000000,1,17,40,0,5,...,0,349.0,349.000000,0.0,4.521739,6.644444,0.0,1.000000,4.000000,0.0
4331,2,44,35,0,0.000000,0,20,40,0,5,...,0,399.0,399.000000,0.0,2.586957,1.733333,0.0,1.500000,1.000000,0.0
4337,2,44,32,4,349.000000,4,20,40,0,5,...,0,349.0,349.000000,0.0,4.521739,6.644444,0.0,6.500000,5.500000,0.0
4377,2,37,35,0,0.000000,0,16,40,0,5,...,0,399.0,399.000000,0.0,2.586957,1.733333,0.0,1.000000,1.000000,0.0
4383,2,37,32,2,349.000000,2,16,40,0,5,...,0,349.0,349.000000,0.0,4.521739,6.644444,0.0,2.000000,2.000000,0.0
4392,2,37,52,0,0.000000,0,16,57,0,7,...,0,0.0,249.000000,0.0,0.108696,0.155556,0.0,0.000000,1.000000,0.0
4423,2,35,35,1,379.049988,1,15,40,0,5,...,0,399.0,399.000000,0.0,2.586957,1.733333,0.0,4.000000,2.000000,0.0
4429,2,35,32,4,344.500000,4,15,40,0,5,...,0,349.0,344.637512,0.0,4.521739,6.644444,0.0,2.000000,5.000000,0.0


In [ ]:
# ['월ID', '상점ID', '상품분류ID']를 기준으로 shop_mean_features 요소별 시차 피처 생성
for shop_mean_feature in shop_mean_features:
    all_data, lag_features_to_clip = add_lag_features(df=all_data,
                                                      lag_features_to_clip=lag_features_to_clip,
                                                      idx_features=['월ID', '상점ID', '상품분류ID'],
                                                      lag_feature=shop_mean_feature,
                                                      nlags=3,
                                                      clip=True)
# shop_mean_features 피처 제거
all_data = all_data.drop(shop_mean_features, axis=1)

In [ ]:
# 시차 피처 생성 마무리 : 결측값 처리
# 월ID 3미만인 데이터 제거
all_data = all_data.drop(all_data[all_data['월ID'] < 3].index)

### 9.4.6 피처 엔지니어링VI: 기타 피처 엔지니어링

In [ ]:
# 기타 피처 추가
all_data['월간 판매량 시차평균'] = all_data[['월간 판매량_시차1',
                                          '월간 판매량_시차2',
                                          '월간 판매량_시차3']].mean(axis=1)

In [ ]:
# 0 ~ 20 사이로 값 제한
all_data[lag_features_to_clip + ['월간 판매량', '월간 판매량 시차평균']] = all_data[lag_features_to_clip + ['월간 판매량', '월간 판매량 시차평균']].clip(0, 20)

In [ ]:
# 시차 변화량
all_data['시차변화량1'] = all_data['월간 판매량_시차1']/all_data['월간 판매량_시차2']
all_data['시차변화량1'] = all_data['시차변화량1'].replace([np.inf, -np.inf],
                                                        np.nan).fillna(0)

all_data['시차변화량2'] = all_data['월간 판매량_시차2']/all_data['월간 판매량_시차3']
all_data['시차변화량2'] = all_data['시차변화량2'].replace([np.inf, -np.inf],
                                                        np.nan).fillna(0)

In [ ]:
# 신상품 여부
all_data['신상여부'] = all_data['첫 판매월'] == all_data['월ID']

In [ ]:
# 첫 판매이후 지난 기간
all_data['첫 판매 후 기간'] = all_data['월ID'] - all_data['첫 판매월']

In [ ]:
all_data['월'] = all_data['월ID'] % 12

In [ ]:
# 첫 판매월, 평균 판매가, 판매건수 피처 제거
all_data = all_data.drop(['첫 판매월', '평균 판매가', '판매건수'], axis=1)

In [ ]:
all_data = downcast(all_data, False) # 데이터 다운캐스팅

### 9.4.7 피처 엔지니어링VII: 마무리

In [ ]:
all_data.info()

In [ ]:
# 훈련 데이터 (피처)
X_train = all_data[all_data['월ID'] < 33]
X_train = X_train.drop(['월간 판매량'], axis=1)
# 검증 데이터 (피처)
X_valid = all_data[all_data['월ID'] == 33]
X_valid = X_valid.drop(['월간 판매량'], axis=1)
# 테스트 데이터 (피처)
X_test = all_data[all_data['월ID'] == 34]
X_test = X_test.drop(['월간 판매량'], axis=1)

# 훈련 데이터 (타깃값)
y_train = all_data[all_data['월ID'] < 33]['월간 판매량']
# 검증 데이터 (타깃값)
y_valid = all_data[all_data['월ID'] == 33]['월간 판매량']

# 가비지 컬렉션
# del all_data
# gc.collect()

### 9.4.8 모델 훈련 및 성능 검증

In [ ]:
import lightgbm as lgb

# LightGBM 하이퍼파라미터
params = {'metric': 'rmse',
          'num_leaves': 255,
          'learning_rate': 0.005,
          'feature_fraction': 0.75,
          'bagging_fraction': 0.75,
          'bagging_freq': 5,
          'force_col_wise': True,
          'random_state': 10}

cat_features = ['상점ID', '도시', '상품분류ID', '대분류', '월']

# LightGBM 훈련 및 검증 데이터셋
dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid)

# LightGBM 모델 훈련
lgb_model = lgb.train(params=params,
                      train_set=dtrain,
                      num_boost_round=1500,
                      valid_sets=(dtrain, dvalid),
                      early_stopping_rounds=150,
                      categorical_feature=cat_features,
                      verbose_eval=100)

### 9.4.9 예측 및 결과 제출

In [ ]:
# 예측
preds = lgb_model.predict(X_test).clip(0, 20)

# 제출 파일 생성
submission['item_cnt_month'] = preds
submission.to_csv(data_path+'submission2.csv', index=False)

In [ ]:
del X_train, y_train, X_valid, y_valid, X_test, lgb_model, dtrain, dvalid
gc.collect();